**Exemplo 02: Previsão de feedbacks de produtos B2W**

Você recebeu um convite para uma consultoria, na qual deve desenvolver um modelo de previsões de feedbacks de clientes em produtos comprados na loja, que serão coletados do instagram.

Os dados que você vai utilizar estão localizados em:
https://raw.githubusercontent.com/americanas-tech/b2w-reviews01/refs/heads/main/B2W-Reviews01.csv

Na coluna 'review_title' você vai encontrar feedbacks passados dos nossos clientes em nossos produtos e, na coluna 'overall_rating', a nota que foi dada. Esse é o único dado que temos para auxiliar na criação desse modelo de previsões.

Dúvidas? Fale comigo!


**Importando bibliotecas**

In [8]:
import pandas as pd
import numpy as np

**Obtendo dados**

In [9]:
# Python
try:
    print('Obtendo dados...')

    # constante dos dados
    ENDERECO_DADOS = 'https://raw.githubusercontent.com/americanas-tech/b2w-reviews01/refs/heads/main/B2W-Reviews01.csv'

    # obtendo dados
    df = pd.read_csv(ENDERECO_DADOS, sep=',', encoding='utf-8')[['review_title','overall_rating']]

    # excluindo dados não existentes (NaN)
    df = df.dropna(subset=['review_title','overall_rating'])

    # Tranformando colunas em arrays
    texts = np.array(df['review_title'])
    rating = np.array(df['overall_rating'])

    print(df.head())
except Exception as e:
    print('Erro ao obter dados: ', e)

Obtendo dados...
                       review_title  overall_rating
0                               Bom               4
1  Preço imbatível, ótima qualidade               4
2      ATENDE TODAS AS EXPECTATIVA.               4
3        presente mais que desejado               4
4            Sem duvidas, excelente               5


C:\Users\36131872024.1\AppData\Local\Temp\ipykernel_25740\2275861674.py:9: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(ENDERECO_DADOS, sep=',', encoding='utf-8')[['review_title','overall_rating']]


**Vetorização**

In [11]:
# Biblioteca para trabalhar com redes neurais artificiais
# Tensorflow - https://www.tensorflow.org/?hl=pt-br
# Tokenizar
from tensorflow.keras.preprocessing.text import Tokenizer # type: ignore
# ajustar o tamanho do vetor
from tensorflow.keras.preprocessing.sequence import pad_sequences # type: ignore

try:
    print('Vetorizando texto...')

    # Passo 1: tokenizar
    tokenizer = Tokenizer()

    # Passo 2: Criar o dicionário
    # fit_on_texts: Cria o vocabulário, através do dicionário
    # associando cada token a um índice numérico
    # lembrando que se a palavra aparecer mais de uma vez, ela vai receber o mesmo índice numérico
    tokenizer.fit_on_texts(texts)

    # Passo 3: Vetorizar, ou seja, transformar os tokens em números,
    # a partir do dicionário criado no passo 2
    vetores = tokenizer.texts_to_sequences(texts)

    # Passo4: Padronização do tamanho do vetor - pad
    padded_vetores = pad_sequences(vetores)

    print(padded_vetores)

    print('Textos vetorizados!')
    
except Exception as e:
    print('Erro ao vetorizar textos: ', e)

Vetorizando texto...
[[   0    0    0 ...    0    0    3]
 [   0    0    0 ... 2620   30   16]
 [   0    0    0 ...  349   45  155]
 ...
 [   0    0    0 ...    0    9    1]
 [   0    0    0 ...    4   19    3]
 [   0    0    0 ...    1    4   51]]
Textos vetorizados!


**Rede Neural**

In [13]:
# Definição do modelo de rede neural utilizada
from tensorflow.keras.models import Sequential # type: ignore
# Camadas da rede neural
from tensorflow.keras.layers import Embedding, LSTM, Dense # type: ignore
# Otimizador de taxa de aprendizado
from tensorflow.keras.optimizers import Adam # type: ignore

try:
    print('Construindo a rede neural...')

    # Constantes do modelo

    # 1ª Constante: Tamanho do vocabulário
    VOCAB_SIZE = len(tokenizer.word_index) + 1

    # 2ª Constante: Tamanho máximo da sequência
    # É o comprimento máximo de um texto
    MAX_SEQUENCE_LENGHT = padded_vetores.shape[1]

    # 3ª Constante: Tamanho do vetor de entrada
    # A literatura recomenda que inicia-se por uma quantidade 
    # igual a raiz quadrada do tamanho do vocabulário
    # Se o volume de dados for de larga escala, pode-se testar iniciando em um tamanho maior
    # Se o volume de dados for muito pequeno, pode-se testar inciando com um tamanho menor
    # Cuidado com o overfitting, que é quando o modelo "aprende demais" e, começa a 
    # perder a capacidade de generalizar melhor, ou seja, obsevar todas as diferenças textuais
    # Overfitting pode ser observado no treino da rede neural
    VETOR_LENGHT = int(np.sqrt(VOCAB_SIZE))

    # Inicia-se a construção da rede neural
    # Sequential é um fluxo linear de camandas (conforme visto na Aula02_RNA.pptx)
    # São processadas em ordem
    model = Sequential()

    # Camada de entrada
    # Embeddings, na qual os vetores de texto são inseridos
    model.add(
        Embedding(
            input_dim=VOCAB_SIZE,
            output_dim=VETOR_LENGHT,
            input_length=MAX_SEQUENCE_LENGHT))

    # Camada oculta ou intermediária
    # LSTM - Long short-term memory, em português "memória de curto e longo prazo"
    # É onde a magia acontece, ou seja, onde o modelo treina baseado nos seus vetores
    # Números de unidades de memória, que basicamente é a qtde de "neurônios"
    # Primeiro TESTE, EXPERIMENTE somente com 1 camada! Cuidado! com o ovberfitting!
    # Se for necessário, adicionar mais camadas, basta repetir o comando abaixo
    
    # primeirca camada oculta
    model.add(LSTM(128))

    # Se necessário, adicionar outra camada oculta
    # model.add(LSTM(64))

    # Camada de saída - Camada Densa
    # Regressão, que é o caso desse exemplo. Somente 1 camada
    model.add(Dense(1))

    # construir o modelo
    # É literamente pegar as definições anteriores e construir o modelo
    # input_shape: é o formato do dados de entrada e ainda o tamanho máximo do texto (MAX_SEQUENCE_LENGHT)
    model.build(input_shape=(None,MAX_SEQUENCE_LENGHT))
    
    # Otimizador de taxa de aprendizado
    # importante para ajustar, em casos de overfitting
    # Adam: É nosso otimizador que vaia ajustar essa taxa de aprendizado
    # learning_rate: Quanto menor, melhor o aprendizado. Menos risco de overfitting
    otimizador = Adam(learning_rate=0.0005)

    # compilar o modelo
    # Verificar se possui algum erro ou se tá de boa
    # Além disso, vamos informar o otimizador e a nossa métrica de perda (loss)
    # loss - Erro quadrado médio (mean_squared_error)
    model.compile(optimizer=otimizador, loss='mean_squared_error')

    model.summary()
    print('Modelo configurado e criado')

except Exception as e:
    print('Erro ao construir a rede neural: ', e)

Construindo a rede neural...


C:\Users\36131872024.1\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 31, 124)        │     1,936,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 128)            │       129,536 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,066,545 (7.88 MB)

 Trainable params: 2,066,545 (7.88 MB)

 Non-trainable params: 0 (0.00 B)

Modelo configurado e criado


In [14]:
# EXPLICAR MELHOR NA PRÓXIMA AULA
from sklearn.model_selection import train_test_split

try:
    print('Treinar o modelo de rede neural')

    X_train, X_test, y_train, y_test = train_test_split(
        padded_vetores,
        rating,
        test_size=0.2,
        random_state=42
    )

    # o treino da rede neural
    model.fit(
        X_train,
        y_train,
        epochs=5,
        batch_size=128,
        validation_data=(X_test, y_test)
    )

except Exception as e:
    print('Erro ao treinar a rede neural: ', e)

Treinar o modelo de rede neural
Epoch 1/5
826/826 ━━━━━━━━━━━━━━━━━━━━ 55s 65ms/step - loss: 1.8788 - val_loss: 0.6964
Epoch 2/5
826/826 ━━━━━━━━━━━━━━━━━━━━ 52s 63ms/step - loss: 0.6222 - val_loss: 0.6617
Epoch 3/5
826/826 ━━━━━━━━━━━━━━━━━━━━ 51s 62ms/step - loss: 0.5393 - val_loss: 0.6590
Epoch 4/5
826/826 ━━━━━━━━━━━━━━━━━━━━ 51s 62ms/step - loss: 0.4970 - val_loss: 0.6512
Epoch 5/5
826/826 ━━━━━━━━━━━━━━━━━━━━ 51s 62ms/step - loss: 0.4691 - val_loss: 0.6681


In [15]:
novos_textos = [
    "Muito bom, gostei bastante. Top demais! Compensa muito!",
    "Não recomendo, péssimo produto. Não funciona"
]

novas_sequencias = tokenizer.texts_to_sequences(novos_textos)
novas_sequencias_padded = pad_sequences(novas_sequencias)

predicoes = model.predict(novas_sequencias_padded)

print("Previsões: ", predicoes)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step
Previsões:  [[ 3.602843  ]
 [-0.04305703]]
